In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("dataset_without.csv")

In [6]:
df=df.drop(columns=['Unnamed: 0'])

In [8]:
# Convert float rating to int class label (0-indexed for BERT classification)
df['label'] = df['Rating'].astype(int) - 1  # Ratings 1–5 -> Labels 0–4

# Remove original Rating column
df = df[['Review', 'label']].rename(columns={'Review': 'text'})

# Split into train and test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
from transformers import AutoTokenizer

#model_name = "microsoft/deberta-v3-small"
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/119640 [00:00<?, ? examples/s]

Map:   0%|          | 0/29911 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

training_args = TrainingArguments(
    output_dir="./bert-review-classifier3",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",
    report_to="none",
    fp16=True
)


In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



/tmp/ipython-input-3333095094.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,1.555800
100,1.388800
150,1.193400
200,1.109800
250,1.068100
300,1.075200
350,1.033800
400,1.043900
450,1.027100
500,1.013700


TrainOutput(global_step=29912, training_loss=0.6658118075946972, metrics={'train_runtime': 2889.4304, 'train_samples_per_second': 165.624, 'train_steps_per_second': 10.352, 'total_flos': 1.573972728219648e+16, 'train_loss': 0.6658118075946972, 'epoch': 4.0})

In [13]:
trainer.save_model('./epoch 4')

In [14]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 4:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 4:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 4: 0.888691073219659
 Training F1 after Epoch 4: 0.8876408389800544


In [15]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 4:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 4:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 4: 0.6504296078365819
 Eval F1 after Epoch 4: 0.6490733845222822


In [16]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 4')

In [17]:

training_args = TrainingArguments(
    output_dir='./bert-review-classifier3',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=1,
    save_strategy="epoch",

    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Resume training from checkpoint
trainer.train(resume_from_checkpoint=True)


/tmp/ipython-input-3703339843.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
29950,0.426300
30000,0.389200
30050,0.464000
30100,0.454800
30150,0.437100
30200,0.429500
30250,0.448000
30300,0.410000
30350,0.434000
30400,0.406600


TrainOutput(global_step=44868, training_loss=0.13084428429635458, metrics={'train_runtime': 1468.3709, 'train_samples_per_second': 488.868, 'train_steps_per_second': 30.556, 'total_flos': 2.360959092329472e+16, 'train_loss': 0.13084428429635458, 'epoch': 6.0})

In [18]:
trainer.save_model('./epoch 6')

In [19]:
train_metrics = trainer.evaluate(eval_dataset=train_dataset)
print(" Training Accuracy after Epoch 6:", train_metrics["eval_accuracy"])
print(" Training F1 after Epoch 6:", train_metrics["eval_f1"])

 Training Accuracy after Epoch 6: 0.9374456703443664
 Training F1 after Epoch 6: 0.9370744848323748


In [20]:
eval_metrics = trainer.evaluate(eval_dataset=test_dataset)
print(" Eval Accuracy after Epoch 6:", eval_metrics["eval_accuracy"])
print(" Eval F1 after Epoch 6:", eval_metrics["eval_f1"])

 Eval Accuracy after Epoch 6: 0.6432081842800308
 Eval F1 after Epoch 6: 0.6429559699066228


**Testing on whole  dataset**

In [ ]:
# testing on whole balanced dataset
new_df = pd.read_csv("dataset.csv")

In [ ]:
new_df = new_df.dropna(subset=['Review'])

In [ ]:
new_df['word_count'] = new_df['Review'].astype(str).apply(lambda x: len(x.split()))

In [ ]:
new_df = new_df[(new_df['word_count'] >= 3) & (new_df['word_count'] <= 200)]

In [ ]:
new_df=new_df.drop(columns=['word_count'])

In [ ]:
new_df=new_df.drop(columns=['Unnamed: 0'])

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149551 entries, 0 to 162815
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  149551 non-null  int64 
 1   Review  149551 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.4+ MB


In [ ]:
new_df['label'] = new_df['Rating'].astype(int) - 1
new_df = new_df[['Review', 'label']].rename(columns={'Review': 'text'})

In [ ]:
new_dataset = Dataset.from_pandas(new_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./epoch 4')

In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

new_dataset = new_dataset.map(tokenize, batched=True)
new_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/149551 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('./epoch 4')

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }


from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    do_train=False,
    do_eval=True,
    report_to="none",
    fp16=True
)

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-678010718.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
from sklearn.metrics import classification_report
target_names = ["Rating 1", "Rating 2", "Rating 3", "Rating 4", "Rating 5"]
predictions_output = trainer.predict(new_dataset)
y_pred = np.argmax(predictions_output.predictions, axis=1)
y_true = predictions_output.label_ids

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4,target_names=target_names))



Classification Report:
              precision    recall  f1-score   support

    Rating 1     0.8995    0.9166    0.9079     27888
    Rating 2     0.7622    0.7252    0.7433     14823
    Rating 3     0.8095    0.8066    0.8081     21670
    Rating 4     0.8526    0.8428    0.8477     38052
    Rating 5     0.9127    0.9263    0.9194     47118

    accuracy                         0.8660    149551
   macro avg     0.8473    0.8435    0.8453    149551
weighted avg     0.8651    0.8660    0.8654    149551



In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Example review text
texts = [
    "I bought this tablet for my 4 year old daughter. Too many advertisements. I would rather pay more and not have a single ad.",

]

# Tokenize
inputs = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
).to(device)

# Prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)+1

print("Predicted labels:", predictions.cpu().numpy())

Predicted labels: [1]
